In [1]:
!pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv(r"dataforecast\raw_data\export-jsps001-1h.csv")# เปลี่ยนเป็นชื่อไฟล์จริง # นำเข้าเอง
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.drop(columns=['timezone'], inplace=True)
df.drop(columns=['pm_2_5_sp'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

# โหลดข้อมูล (สมมติว่ามีคอลัมน์ 'timestamp')
df['timestamp'] = pd.to_datetime(df['timestamp'])  # แปลงเป็น datetime
df

,timestamp,humidity,pm_2_5,temperature
0,2021-08-15 07:00:00,89.789247,29.242424,26.725806
1,2021-08-15 08:00:00,88.830000,25.800000,27.213333
2,2021-08-15 09:00:00,86.617241,19.655172,27.627586
3,2021-08-15 10:00:00,84.392453,34.826923,28.116981
4,2021-08-15 11:00:00,83.094167,55.907563,28.509167
...,...,...,...,...
27544,2025-02-11 08:00:00,87.340026,43.263158,23.013561
27545,2025-02-11 09:00:00,80.816273,42.478992,25.590026
27546,2025-02-11 10:00:00,67.041454,40.848214,29.023077
27547,2025-02-11 11:00:00,53.353909,31.842105,32.314087


In [23]:
df.boxplot()

<Axes: >

In [24]:
import numpy as np
from sklearn.ensemble import IsolationForest
features = ["pm_2_5", "humidity", "temperature"]
df_subset = df[features]
# ใช้ Isolation Forest ตรวจจับ Outlier
iso = IsolationForest(contamination=0.05, random_state=42)
outlier_pred = iso.fit_predict(df_subset)
# ตำแหน่งที่เป็น Outlier จะมีค่า -1
df["outlier"] = outlier_pred
# แทนที่ Outlier ด้วยค่ามัธยฐานของคอลัมน์นั้น
for col in features:
    median_value = df[col].median()
    df.loc[df["outlier"] == -1, col] = median_value

df.drop(columns=["outlier"], inplace=True)
df

,timestamp,humidity,pm_2_5,temperature
0,2021-08-15 07:00:00,89.789247,29.242424,26.725806
1,2021-08-15 08:00:00,88.830000,25.800000,27.213333
2,2021-08-15 09:00:00,86.617241,19.655172,27.627586
3,2021-08-15 10:00:00,84.392453,34.826923,28.116981
4,2021-08-15 11:00:00,83.094167,55.907563,28.509167
...,...,...,...,...
27544,2025-02-11 08:00:00,87.340026,43.263158,23.013561
27545,2025-02-11 09:00:00,80.816273,42.478992,25.590026
27546,2025-02-11 10:00:00,67.041454,40.848214,29.023077
27547,2025-02-11 11:00:00,53.353909,31.842105,32.314087


In [ ]:
df['date'] = df['timestamp'].dt.date  # ดึงเฉพาะวันที่

# นับจำนวนแถวต่อวัน
count_per_day = df.groupby('date')['timestamp'].count()

# คัดกรองเฉพาะวันที่มีครบ 24 ชั่วโมง
valid_days = count_per_day[count_per_day == 24].index

# ลบวันที่ไม่ครบ 24 ชั่วโมง
df = df[df['date'].isin(valid_days)].drop(columns=['date'])

df.to_csv(r"dataforecast\cleandata_hours.csv", index=False) #ส่งออกเองชื่อ clean_hours โดยเป็นไฟล์ที่ยังไม่ลบ 7 ัวน
df



,timestamp,humidity,pm_2_5,temperature
45,2021-08-18 00:00:00,82.810924,6.478261,27.261345
46,2021-08-18 01:00:00,84.977966,14.271186,26.650000
47,2021-08-18 02:00:00,84.600000,32.308333,26.710924
48,2021-08-18 03:00:00,85.113559,29.743590,26.647458
49,2021-08-18 04:00:00,85.430833,38.931034,26.551667
...,...,...,...,...
27531,2025-02-10 19:00:00,69.219469,27.588235,28.891425
27532,2025-02-10 20:00:00,70.651656,26.825000,28.379455
27533,2025-02-10 21:00:00,71.649910,25.441667,27.974364
27534,2025-02-10 22:00:00,71.973936,26.056075,27.943536


In [ ]:
df = df.iloc[:-168]  # ลบแถวท้าย N แถว
df.to_csv(r"dataforecast\last_test.csv", index=False) #ส่งออกเองชื่อ last_test ไฟล์หลังลบ 7 วัน
df

,timestamp,humidity,pm_2_5,temperature
45,2021-08-18 00:00:00,82.810924,6.478261,27.261345
46,2021-08-18 01:00:00,84.977966,14.271186,26.650000
47,2021-08-18 02:00:00,84.600000,32.308333,26.710924
48,2021-08-18 03:00:00,85.113559,29.743590,26.647458
49,2021-08-18 04:00:00,85.430833,38.931034,26.551667
...,...,...,...,...
27363,2025-02-03 19:00:00,70.578723,28.591667,29.206171
27364,2025-02-03 20:00:00,69.189982,29.714286,28.579493
27365,2025-02-03 21:00:00,72.229974,22.954545,27.387404
27366,2025-02-03 22:00:00,74.487035,21.478992,26.806460
